# Curve Fitting
code from michjw. 

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.interpolate import *
import matplotlib.pyplot as plt
import pandas as pd
import datetime, math
import hvplot.pandas
import holoviews as hv
from statsmodels import robust
hv.extension('bokeh')

In [ ]:
df_results = pd.read_pickle('/home/jovyan/data/camhd_data/regions_results_for_dep_2.pickle')
df_results.head()

In [ ]:
# codemaderaw for use in emilio's enviorment
df_results = pd.read_pickle('/home/jovyan/camhd_floc_model/data_camhd/results_for_dep_2.pickle')
#total_floc = df_results['total_floc']
df_results.head()

In [ ]:
df_results = df_results.loc[df_results['nflocs']<2000]
plot1= df_results.hvplot.scatter('datetime', 'total_floc',dynspread=True, datashade=True)
plot1

In [ ]:
time = df_results['timestamp'].values
nfloc = df_results['nflocs'].values
datedate = df_results['datetime']

In [ ]:
def func (time, a1, a2):
    return a1 + a2 * np.log10(time)

In [ ]:
popt, pcov = curve_fit(func, df_results['timestamp'], df_results['nflocs'])
y2 = func(time, popt[0], popt[1])

In [ ]:
df_results['y2'] = y2
plot1 = df_results.hvplot.scatter('datetime','nflocs',dynspread=True, datashade=True)
plot2 = df_results.hvplot.line('datetime', 'y2',color='orange')
plot1*plot2

# Use Median Bins

In [ ]:
# Get weekly median # flocs
nflocs_med = df_results.set_index('datetime').resample('W')["nflocs"].median()
# create a dataframe with median nflocs (with weekly datetime value as index)
df = nflocs_med.to_frame('nf_med')
# Plot
plot1 = df_results.hvplot.scatter('datetime','nflocs',dynspread=True, datashade=True)
plot2 = df.hvplot.scatter('datetime','nf_med', color = 'red')
plot1*plot2

In [ ]:
xx = np.arange(1,len(df)+1)
df['xx'] = xx
df = df.dropna()

popt, pcov = curve_fit(func, df['xx'], df['nf_med'])
y2 = func(df['xx'], popt[0], popt[1])
y2 = y2.copy() # doing this to avoid warning about copying a df slice
df['y2'] = y2

# Plot
plot1 = df_results.hvplot.scatter('datetime','nflocs',dynspread=True, datashade=True,color='lightblue')
plot2 = df.hvplot.scatter('datetime','nf_med', color = 'red')
plot3 = df.hvplot.line('datetime','y2',color = 'gray')
plot1*plot3*plot2